# 第4章 Pandas统计分析基础

Pandas（Python Data Analysis Library）是基于NumPy的数据分析模块，它提供了大量标准数据模型和高效操作大型数据集所需的工具，可以说Pandas是使得Python能够成为高效且强大的数据分析环境的重要因素之一。

导入方式：import pandas as pd

## 4.1 Pandas中的数据结构

Pandas有三种数据结构：Series、DataFrame和Panel。  
Series类似于一维数组；DataFrame是类似表格的二维数组；Panel可以视为Excel的多表单Sheet

### 4.1.1 Series
Series 是一种一维数组对象，包含了一个值序列，并且包含了数据标签，称为索引（index），可通过索引来访问数组中的数据。

1. Series的创建  
1) 通过列表创建

【例4-1】通过列表创建Series。

In [293]:
import pandas as pd
obj = pd.Series([1, -2, 3, -4]) #仅有一个数组构成
print(obj)
print("通过索引访问：",obj[2])
print("通过切片访问：\n",obj[:2])
print(type(obj[:2]))

0    1
1   -2
2    3
3   -4
dtype: int64
通过索引访问： 3
通过切片访问：
 0    1
1   -2
dtype: int64
<class 'pandas.core.series.Series'>


输出的第一列为index,第二列为数据value。如果创建 Series 时没有指定 index，Pandas会果用**整型数据**作为该 Series 的 index，也可以使用 Python 里的索引 index 和切片 slice 技术。

【例4-2】创建Series时指定索引。

In [294]:
i = ("a", "c", "d", "a")
v = [2, 4, 5, 7]
t = pd.Series(v, index = i, 
              name = "col")
print(t)
print(t["a"])
print(t[0])

a    2
c    4
d    5
a    7
Name: col, dtype: int64
a    2
a    7
Name: col, dtype: int64
2


尽管创建Series指定了index参数，实际Pandas还是有隐藏的index位置信息的。  
所以Series有两套描述某条数据的手段：**位置和标签**

【例4-3】Series位置和标签的使用。

In [295]:
val = [2, 4, 5, 6]
idx1 = range(10, 14)
idx2 = "hello the cruel world".split()
s0 = pd.Series(val)
s1 = pd.Series(val, index = idx1)
t = pd.Series(val, index = idx2)
print(s0.index)
print(s1.index)
print(t.index)

RangeIndex(start=0, stop=4, step=1)
RangeIndex(start=10, stop=14, step=1)
Index(['hello', 'the', 'cruel', 'world'], dtype='object')


In [298]:
# val = [2, 4, 5, 6]
print(s0[0])
print(s1[10])
# print(s1[0])   #输出是什么？
print('default:',t[0],'label:' ,t["hello"])

2
2
default: 2 label: 2


2） 通过字典创建  
如果数据被存放在一个Python字典中，也可以直接通过这个字典来创建Series。   
如果只传入一个字典，则结果Series中的索引就是原字典的键。

【例4-4】通过字典创建Series。

In [299]:
sdata = {'Ohio': 35000, 'Texas': 71000, 
         'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
print(obj3)
print(obj3["Ohio"])    #以下两句的输出是否一致。
print(obj3[0])

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64
35000
35000


【例4-5】通过字典创建Series时的索引。

In [ ]:
sdata = {"a" : 100, "b" : 200,
         "e" : 300}
obj3 = pd.Series(sdata)
print(obj3)

如果字典中的键值和指定的索引不匹配，则对应的值是NaN.

【例4-6】键值和指定的索引不匹配。

In [300]:
sdata = {"a" : 100, "b" : 200, 
         "d": 250,"e" : 300}
letter = ["a", "b", "c", "e" ,"h"]
obj =  pd.Series(sdata, index = letter)
print(obj)

a    100.0
b    200.0
c      NaN
e    300.0
h      NaN
dtype: float64


对于许多应用而言，Series重要的一个功能是：**它在算术运算中会自动对齐不同索引的数据。**

【例4-7】不同索引数据的自动对齐。

In [302]:
sdata = {'Ohio': 35000, 'Texas': 71000,
         'Oregon': 16000, 'Utah': 5000}
obj1 = pd.Series(sdata)
print(obj1)

states = ['California', 'Ohio',
          'Oregon', 'Texas']
obj2 = pd.Series(sdata, index = states)
print(obj2)

print(obj1+obj2)

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64
California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64


**Series的索引可以通过赋值的方式就地修改。**

【例4-8】Series索引的修改。

In [305]:
obj = pd.Series([4,7,-3,2])
print("修改前：\n",obj)

obj.index = ['Bob', 'Steve', 
             'Jeff', 'Ryan']   #是否能增加索引项
print("修改后：\n",obj)

修改前：
 0    4
1    7
2   -3
3    2
dtype: int64
修改后：
 Bob      4
Steve    7
Jeff    -3
Ryan     2
dtype: int64


In [309]:
# 数据的描述信息
print(obj.describe())
obj.sum() #常用方法

count    4.000000
mean     2.500000
std      4.203173
min     -3.000000
25%      0.750000
50%      3.000000
75%      4.750000
max      7.000000
dtype: float64


10

### 4.1.2 DataFrame
DataFrame是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值、字符串、布尔值等）。  
DataFrame既有**行索引也有列索引**，它可以被看做由Series组成的字典（共用同一个索引）。


**DataFrame的创建**  
构建 DataFrame 的方式有很多,最常用的是直接传入一个由等长列表或 NumPy 数组组成的字典来形成 DataFrame。

【例4-9】DataFrame的创建。

In [311]:
data = {
    'name':['张三', '李四', '王五', '小明'],
    'sex':['female', 'female', 'male', 'male'],
    'year':[2001, 2001, 2003, 2002],
    'city':['北京', '上海', '广州', '北京']
}
df = pd.DataFrame(data)
display(df)
# print(df)

,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


  name     sex  year city
0   张三  female  2001   北京
1   李四  female  2001   上海
2   王五    male  2003   广州
3   小明    male  2002   北京


DataFrame 会自动加上索引(跟 Series 一样)，且全部列会被有序排列。   
如果指定了列名序列，则 DataFrame 的列就会按照指定顺序进行排列。

【例4-10】DataFrame的索引。

In [312]:
df1 = pd.DataFrame(data, 
    columns = ['name', 'year', 'sex', 'city'])  #是否能增加列名
display(df1)

,name,year,sex,city
0,张三,2001,female,北京
1,李四,2001,female,上海
2,王五,2003,male,广州
3,小明,2002,male,北京


跟Series一样，如果传入的列在数据中找不到，就会产生NaN值

【例4-11】DataFrame创建时的空缺值。

In [313]:
df2 = pd.DataFrame(data, 
    columns = ['name', 'year', 
    'sex', 'city','address'])
display(df2)

,name,year,sex,city,address
0,张三,2001,female,北京,NaN
1,李四,2001,female,上海,NaN
2,王五,2003,male,广州,NaN
3,小明,2002,male,北京,NaN


DataFrame构造函数的columns函数给出列的名字，index给出label标签。  

【例4-12】DataFrame创建时指定列名。

In [314]:
df3 = pd.DataFrame(data, 
    columns = ['name', 'sex', 'year', 'city'], 
    index = ['a', 'b', 'c', 'd'])
display(df3) 

,name,sex,year,city
a,张三,female,2001,北京
b,李四,female,2001,上海
c,王五,male,2003,广州
d,小明,male,2002,北京


### 4.1.3  索引对象  
Pandas的索引对象负责管理轴标签和其他元数据（比如轴名称等）。  
构建Series或 DataFrame时，所用到的任何数组或其他序列的标签都会被转换成一个Index。

【例4-13】显示DataFrame的索引和列。

In [315]:
print(df3)
print(df3.index)
print(df3.columns)
print(df3.columns[0])

  name     sex  year city
a   张三  female  2001   北京
b   李四  female  2001   上海
c   王五    male  2003   广州
d   小明    male  2002   北京
Index(['a', 'b', 'c', 'd'], dtype='object')
Index(['name', 'sex', 'year', 'city'], dtype='object')
name


**索引对象不能进行修改**，否则会报错，因此用户不能对其进行修改，不可修改性非常重要，因为这样才能使 Index 对象在多个数据结构之间安全共享。  

除了长得像数组，index 的功能也类似于一个固定大小的集合。 

In [318]:
display(df3)
# df3.index[0] = 'z'
df3.index = ['e','f','g','h']
display(df3)

,name,sex,year,city
e,张三,female,2001,北京
f,李四,female,2001,上海
g,王五,male,2003,广州
h,小明,male,2002,北京


,name,sex,year,city
e,张三,female,2001,北京
f,李四,female,2001,上海
g,王五,male,2003,广州
h,小明,male,2002,北京


【例4-14】DataFrame的index

In [ ]:
print('name' in df3.columns)
print('a' in df3.index)

每个索引都有一些方法和属性，它们可用于设置逻辑并回答有关该索引所包含的数据的常见问题。Index的常用方法和属性见表4-1。

In [319]:
%%html
<img src = "tables\table4-1.png",width=500, height=100>

【例4-15】插入索引值。

In [320]:
display(df)
print(df.index)
print(df.index.insert(1,'w'))  #得到新的索引
display(df)

,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


RangeIndex(start=0, stop=4, step=1)
Index([0, 'w', 1, 2, 3], dtype='object')


,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


In [323]:
newIndex = df.index.insert(1,'w')  #得到新的索引
display(newIndex)
display(df)
# df.index = newIndex  #是否可行
# df.index = ['e','f','f','h']
# print(df.index)
# print(df)

Index([0, 'w', 1, 2, 3], dtype='object')

,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


### 4.1.4 查看DataFrame的常用属性 
DataFrame的基础属性有values、index、columns、dtypes、ndim和shape，分别可以获取DataFrame的元素、索引、列名、类型、维度和形状

【例4-16】显示DataFrame的属性。

In [324]:
display(df)

,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


In [325]:
print('信息表的所有值为：\n',df.values)
print(type(df.values))
print('信息表的所有列为：\n',df.columns)
print(type(df.columns))
print('信息表的元素个数为：\n ',df.size)
print('信息表的维度是\n ',df.ndim)
print('信息表的形状为：',df.shape)

信息表的所有值为：
 [['张三' 'female' 2001 '北京']
 ['李四' 'female' 2001 '上海']
 ['王五' 'male' 2003 '广州']
 ['小明' 'male' 2002 '北京']]
<class 'numpy.ndarray'>
信息表的所有列为：
 Index(['name', 'sex', 'year', 'city'], dtype='object')
<class 'pandas.core.indexes.base.Index'>
信息表的元素个数为：
  16
信息表的维度是
  2
信息表的形状为： (4, 4)


## 4.2 Pandas索引操作
### 4.2.1 重新索引
索引对象是无法修改的，因此，**重新索引是指对索引重新排序而不是重新命名**，如果某个索引值不存在的话，会引入缺失值。

【例4-17】重建索引

In [327]:
obj = pd.Series([7.2,-4.3,4.5,3.6],
        index = ['b', 'a', 'd', 'c'])
display(obj)
display(obj.reindex(['a','b',
                     'c','d','e']))
display(obj)   #obj是否变了？

b    7.2
a   -4.3
d    4.5
c    3.6
dtype: float64

a   -4.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

b    7.2
a   -4.3
d    4.5
c    3.6
dtype: float64

In [329]:
obj = obj.reindex(['a','b','c','d','e'])
display(obj)

a   -4.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对于重建索引引入的缺失值，可以使用fill_value参数填充。

【例4-18】重建索引时填充缺失值。

In [335]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'],
                   fill_value = 0)
display(obj2)

a   -4.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对于顺序数据，比如时间序列，重新索引时可能需要进行插值或填值处理，利用参数method选项可以设置：  
method = ‘ffill’或‘pad’，表示前向值填充  
method = ‘bfill’或‘backfill’，表示后向值填充

【例4-19】缺失值的前向填充。

In [336]:
import numpy as np
obj1 = pd.Series(['blue','red','black'],
                 index = [0,2,4])
print(obj1)

0     blue
2      red
4    black
dtype: object


In [339]:
obj1.reindex(np.arange(6),
             method = 'bfill',fill_value=0)

0     blue
1      red
2      red
3    black
4    black
5        0
dtype: object

【例4-20】缺失值的后向填充。

In [ ]:
obj2 = pd.Series(['blue','red','black'],
                 index = [0,2,4])
obj2.reindex(np.arange(6),
             method = 'backfill')
# display(obj2)

**对于DataFrame,reindex 可以修改行(列)索引，或两个都修改。**   
如果仅传入一个序列，则结果中的行会重建索引

【例4-21】【例4-22】reindex操作。

In [340]:
import numpy as np
df4 = pd.DataFrame(np.arange(9).reshape(3,3),
    index = ['a','c','d'],
    columns = ['one','two','four'])
print(df4)

   one  two  four
a    0    1     2
c    3    4     5
d    6    7     8


In [344]:
df4.reindex(index = ['a','b','c','d'],
            columns = ['one','two','three','four'])
display(df4)

,one,two,four
a,0,1,2
c,3,4,5
d,6,7,8


传入fill_value = n，用n代替缺失值。

【例4-23】参数fill_value的用法。

In [ ]:
df5 = df4.reindex(index = ['a','b','c','d'],
    columns = ['one','two','three','four'],
    fill_value = 100)
display(df4)
display(df5)

表4-2. reindex函数参数

In [345]:
%%html
<img src = "tables\table4-2.png",width=500, height=100>

### 4.2.2 更换索引
如果不希望使用默认的行索引，则可以在创建的时候通过index参数来设置。  
在DataFrame数据中，**如果希望将列数据作为索引**，则可以通过set_index方法来实现。   


【例4-24】重建索引。

In [346]:
display(df)
df3 = df.set_index('city')
display(df3)
display(df)

,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


,name,sex,year
city,,,
北京,张三,female,2001
上海,李四,female,2001
广州,王五,male,2003
北京,小明,male,2002


,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


## 4.3 DataFrame数据的查询与编辑 

### 4.3.1 DataFrame数据的查询
在数据分析中，选取需要的数据进行分析处理是最基本操作。在Pandas中需要通过索引完成数据的选取。 

**1. 选取列:**    

通过列索引或以属性的方式可以单独获取DataFrame的列数据，返回的数据类型为Series。    

此外，也可以通过数据类型选择columns。   

在选取列时注意**不能使用切片方式。**


【例4-25】选取列数据。

In [347]:
display(df)

,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


In [348]:
w1 = df['year']
print('选取1列数据：\n',w1)
print('平均年份：',w1.mean())

选取1列数据：
 0    2001
1    2001
2    2003
3    2002
Name: year, dtype: int64
平均年份： 2001.75


In [ ]:
print(type(w1))
print(w1[1])

In [352]:
w2 = df[['name',"year"]]   #该访问方式正确吗？
display('选取2列数据:',w2)
print(type(w2))
# print(w2[1])    #以下的访问方式正确吗？
# print(w2[1][1])

'选取2列数据:'

,name,year
0,张三,2001
1,李四,2001
2,王五,2003
3,小明,2002


<class 'pandas.core.frame.DataFrame'>


In [353]:
display(df.select_dtypes(exclude='int64').head())

,name,sex,city
0,张三,female,北京
1,李四,female,上海
2,王五,male,广州
3,小明,male,北京


In [354]:
display(df.select_dtypes(include=['int64','object']).head())

,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


**2. 选取行**  

通过行索引或行索引位置的切片形式可以选取行数据。

【例4-26】选取行数据。

In [ ]:
display(df)

In [357]:
display('显示指定的行:',df[1:3])  
print(type(df[1:2]),"\n")
# display('显示2-3两行:',df[1:3])  

'显示指定的行:'

,name,sex,year,city
1,李四,female,2001,上海
2,王五,male,2003,广州


<class 'pandas.core.frame.DataFrame'> 



通过DataFrame提供的head和tail方法可以得到多行数据，但是用这两种方法得到的数据都是从开始或者末尾获取连续的数据，而利用sample可以随机抽取数据并显示。  
head（） #默认获取前5行  
head（n）#获取前n行  
tail（）#默认获取后5行  
tail（n）#获取后n行  
sample（n）#随机抽取n行显示
sample（frac = 0.6）#随机抽取60%的行显示

In [360]:
display(df.head())
display(df.sample(2))

,name,sex,year,city
0,张三,female,2001,北京
1,李四,female,2001,上海
2,王五,male,2003,广州
3,小明,male,2002,北京


,name,sex,year,city
3,小明,male,2002,北京
2,王五,male,2003,广州


**3. 选取行和列**  
DataFrame.loc[行索引名称或条件，列索引名称]  
DataFrame.iloc[行索引位置，列索引位置]


【例4-27】loc选取行和列。

In [361]:
display(df3)

,name,sex,year
city,,,
北京,张三,female,2001
上海,李四,female,2001
广州,王五,male,2003
北京,小明,male,2002


In [362]:
display(df3.loc[:,['name','year']]) 
#显示name和year两列

,name,year
city,,
北京,张三,2001
上海,李四,2001
广州,王五,2003
北京,小明,2002


In [363]:
display(df3.loc[['北京','上海'],['name','year']] ) 
#显示北京和上海行中的name和year两列

,name,year
city,,
北京,张三,2001
北京,小明,2002
上海,李四,2001


In [364]:
display(df3.loc[df3['year']>=2002,
                ['name','year']] ) 

,name,year
city,,
广州,王五,2003
北京,小明,2002


In [365]:
display(df3.loc[df3['year'].isin(['2001','2003']),
                ['name','year']] ) 

,name,year
city,,
北京,张三,2001
上海,李四,2001
广州,王五,2003


【例4-28】利用iloc选取行和列。

In [366]:
display(df3)
df4 = df3.iloc[:,2]
display(df4) 
display(type(df4))

,name,sex,year
city,,,
北京,张三,female,2001
上海,李四,female,2001
广州,王五,male,2003
北京,小明,male,2002


city
北京    2001
上海    2001
广州    2003
北京    2002
Name: year, dtype: int64

pandas.core.series.Series

In [368]:
display(df3)
display(df3.iloc[2,[1,2]] ) #显示前两列
# display(type(df3.iloc[2,[1,2]]))
df3.iloc[2,[1,2]]['sex']

,name,sex,year
city,,,
北京,张三,female,2001
上海,李四,female,2001
广州,王五,male,2003
北京,小明,male,2002


sex     male
year    2003
Name: 广州, dtype: object

'male'

In [ ]:
display(df3.iloc[[1,3],2]) #显示第1和第3行
display(type(df3.iloc[[1,3],2]))
# display(df3.iloc[[1,3],[1,2]])

【例4-29】利用query查询数据。

In [ ]:
display(df3.query("year > 2001"))
display(df3.query("year > 2001 & year < 2003"))

**4. 布尔选择**  
可以对DataFrame中的数据进行布尔方式选择。

【例4-30】布尔选择。

In [ ]:
display(df3)

In [ ]:
df4 = df3[df3['year']==2001]
display(df4)

In [ ]:
df4 = df3[(df3.year == 2001)]
display(df4)

In [ ]:
df4 = df3[(df3.year > 2001) & (df3.year < 2003)]
display(df4)

In [ ]:
display(df4)
display(df4.loc["北京"])    # 以下访问的结果一样吗？
# display(type(df4.loc["北京"]))
# display(df4.loc[["北京"]])
# display(type(df4.loc[["北京"]]))

In [ ]:
display(df4['北京'])     #两句代码效果一样吗？
# display(df4.loc['北京'])

In [ ]:
df4 = df3[df3['year']==2001]
display(df4.loc[["北京"]])
display(df3.loc[["北京"]])

In [ ]:
df = pd.DataFrame({'price':[1.99,3,5,0.5,3.5,5.5,3.9]})
display(df)
display(df[(df.price>=2) & (df.price<=4)])
display(df[(df["price"]>=2) & (df["price"]<=4)])
# df[df.price.between(2,4)]

### 4.3.2 DataFrame数据的编辑  
编辑DataFrame中的数据是将需要编辑的数据提取出来，重新赋值。  
**1. 增加数据**  
增加一行直接通过append方法传入字典结构数据即可。

【例4-31】增加一行数据。

In [ ]:
display(df3)

In [ ]:
data1 = {'city':'兰州','name':'李红',
    'year':2005,'sex':'female'}
# df4 = df3.append(data1,ignore_index = True)
df4 = df3.append(data1)
display(df4)
display(df3)

In [ ]:
data1 = {'city':'兰州','name':'李红',
         'year':2005,'sex':'female'}
s = pd.Series(data1,name = "test")
# print(s)
display(df3.append(s))

增加列时，只需为要增加的列赋值即可创建一个新的列。

【例4-32】增加一列并赋值。

In [ ]:
# display(df3)
df3['score'] = [85,78,96,80]
display(df3)

In [ ]:
df3.insert(1,'no',['001','002','003','004'])
display(df3)

**2. 删除数据**  
删除数据直接用drop方法，通过**axis参数**确定是删除的是行还是列。  
**默认数据删除不修改原数据**，需要在原数据删除行列需要设置**参数inplace = True**。


【例4-33】删除数据行。

In [ ]:
display(df3.drop("广州"))
display(df3)

【例4-34】删除数据的列。

In [ ]:
df3.drop('sex',axis = 1,inplace = True)
display(df3)

**3. 修改数据**  
修改数据时直接对选择的数据赋值即可。  
需要注意的是，数据修改是直接对DataFrame数据修改，操作无法撤销，因此更改数据时要做好数据备份。


In [ ]:
display(df3)

In [ ]:
df3['score'] = [100,100,96,80]
# df3.iloc[3,3]= 100
display(df3)

In [ ]:
df3.replace({"张三":"zhang"})

4. 修改列名
Pandas通过DataFrame.rename()函数，传入需要修改列名的字典形式来修改列名。   

【例4-34】修改数据表的列名

In [ ]:
df3.rename(columns = {'score':'number'},inplace=True)
display(df3)

## 4.4 Pandas数据运算
### 4.4.1 算术运算


Pandas的数据对象在进行算术运算时，如果有相同索引则进行算术运算，如果没有，则会自动进行数据对齐，但会引入缺失值。

【例4-35】Series相加。

In [ ]:
obj1 = pd.Series([5.1,-2.6,7.8,10],
    index = ['a','c','g','f'])
print('obj1:\n',obj1)
obj2 = pd.Series([2.6,-2.8,3.7,-1.9],
    index = ['a','b','g','h'])
print('obj2:\n',obj2)

In [ ]:
print(obj1+obj2)

对于DataFrame，数据对齐操作会同时发生在行和列上。

【例4-36】DataFrame类型的数据相加。

In [ ]:
a = np.arange(6).reshape(2,3)
b = np.arange(4).reshape(2,2)
df1 = pd.DataFrame(a,columns = ['a','b','e'],
    index = ['A','C'])
display('df1:\n',df1)

df2 = pd.DataFrame(b,columns = ['a','b'],
    index = ['A','D'])
display('df2:\n',df2)

In [ ]:
display('df1+df2:\n',df1+df2)

### 4.4.2 函数应用和映射  
在数据分析时，经常会需要对数据进行较复杂的运算，此时需要定义函数。
已定义好的函数可以通过以下三种方法应用到数据：

1. map函数：将函数套用到Series的每个元素中；
2. apply函数，将函数套用到DataFrame的行或列上，行与列通过axis参数设置；
3. applymap函数，将函数套用到DataFrame的每个元素上。


【例4-37】将水果价格表中的“元”去掉。

In [ ]:
data = {'fruit':['apple','grape','banana'],
        'price':['30元','43元','28元']}
df1 = pd.DataFrame(data)
display(df1)

In [ ]:
def f(x):
    print(x)
    return x.split('元')[0]

df1['price'] = df1['price'].map(f)
display('修改后的数据表:\n',df1)

【例4-38】apply函数的使用方法。

In [ ]:
df2 = pd.DataFrame(np.random.randn(3,3),
                columns = ['a','b','c'],
                index = ['app','win','mac'])
display(df2)

In [ ]:
df2.apply(np.max,axis=0)

In [ ]:
def f(x):
    print(x)
    print(type(x))
#     price = x[1].split('元')[0]
#     if x[0]=="apple":
#         return int(price)+5
#     else:
#         return int(price)-5
df1['new price'] = df1.apply(f,axis=0)
display('修改后的数据表:\n',df1)

【例4-39】applymap函数的用法。

In [ ]:
display(df2)
def f(x):
    return 2*x
display(df2.applymap(f))

### 4.4.3 排序  
sort_index方法：对索引进行排序，**默认为升序**，降序排序时加参数 ascending=False。  
sort_values方法：对数值进行排序。by参数设置待排序的列名


【例4-40】Series的排序。

In [ ]:
import pandas as pd
wy = pd.Series([1,-2,4,-4],
               index = ['c','b','a','d'])
print(wy)
print('排序后的Series:\n',wy.sort_index())
# print(wy) #是否改变wy

【例4-41】对Series的数值排序。

In [ ]:
print('值排序后的Series:\n',wy.sort_values())
# print(wy)#是否改变wy

对于DataFrame数据排序，通过指定轴方向，使用sort_index函数对**行或列索引**进行排序。  
如果要根据**某列值进行排序**，则通过sort_values函数把列名传给by参数即可。

【例4-42】DataFrame排序。

In [ ]:
display(df2)

In [ ]:
df2.sort_index(axis=1,ascending = False)

In [ ]:
display(df2)
df2.sort_values(by = 'b')

In [ ]:
display(df2)
df2.sort_values(by = 'app',axis=1,ascending = False)

### 4.4.4 汇总与统计
**1. 数据汇总**  
在DataFrame中，可以通过sum方法对每列进行求和汇总，与Excel中的sum函数类似。  
如果设置axis = 1指定轴方向，可以实现按行汇总。


【例4-43】DataFrame中的汇总。

In [ ]:
display(df2)

In [ ]:
display('按列汇总:\n',df2.sum())
display('按行汇总:\n',df2.sum(axis = 1))

**2.数据描述与统计**  
描述性统计用来概括、表述事物整体状况以及事物间关联、类属关系的统计方法。通过一些统计值可以描述一组数据的集中趋势和离散程度等分布状态。  
利用describe方法可以对每个**数值型列**进行统计。


【例4-44】describe示例。

In [ ]:
display(df2)
df2.describe()
df2.value_counts()

In [ ]:
%%html
<img src = "tables\table4-3.png", width=500, height=80>

对于类别型特征的描述性统计，可以使用频数统计表。  
Pandas库中通过unique方法获取不重复的数组，Nunique方法计算行或列上唯一值的数量，利用value_counts方法实现频数统计。

【例4-45】数据的频数统计。

In [ ]:
np.random.seed(971)
value_1 = np.random.randint(10,size=8)
value_2 = np.random.randint(10,size=8)
years = np.arange(2010,2018)
groups = ['A','A','B','A','B','C','A','C']
df = pd.DataFrame({'group':groups,'year':years,
        'value_1':value_1,'value_2':value_2})
display(df)
print('group的取值为:',df.group.unique())
print('group的取值个数为:',df.group.nunique())
print('value_1的取值及其个数为:\n',df.value_1.value_counts())


### 4.5 数据分组与聚合
数据分组与聚合的思想来源于关系型数据库，是一类重要的数据操作。
### 4.5.1 数据分组
根据某个或某几个字段对数据集进行分组，然后对每个分组进行分析与转换，是数据分析中常见的操作。  
Pandas提供了一个高效的 groupby 方法，配合 agg或 apply 方法实现数据分组聚合的操作。

**1.groupby方法**  
groupby方法可以根据索引或字段对数据进行分组。  
格式为：
DataFrame.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False)

对于参数 by，如果传入的是一个函数，则对**索引进行计算**并分组；  
如果传入的是字典或 Series，则字典或 Series 的**值**作为分组依据；  
如果传入的是 NumPy数组,则**数据元素**作为分组依据;  
如果传入的是字符串或字符串列表，则用这些**字符串所代表的字段**作为分组依据

【例4-46】groupby基本用法举例。

In [ ]:
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
    'key2' : ['yes', 'no', 'yes', 'yes', 'no'],
    'data1' : np.random.randn(5),
    'data2' : np.random.randn(5)})
display(df)

In [ ]:
#如果传入的是Series，则Series 的值作为分组依据；  
grouped = df["data1"].groupby(df['key1'])
display(grouped)

In [ ]:
print(grouped.size())
print(grouped.mean())

数据分组后返回数据的数据类型，它不再是一个DataFrame，而是一个 groupby对象。  
可以调用 groupby 的方法，如 size 方法，返回一个含有分组大小的 series; mean 方法，返回每个分组数据的均值。

**2.按列名分组**  
groupby方法使用的分组键除了是Series,也可以是其他的格式。  
DataFrame 数据的**列索引名**可以作为分组键，但需要注意的是，用于分组的对象**必须是DataFrame数据本身**,否则搜索不到索引名称会报错。

【例4-47】列索引名称作为分组键。

In [ ]:
display(df)

In [ ]:
#传入的是字符串或字符串列表，则用这些字符串所代表的字段作为分组依据
display(df.groupby('key2').mean()['data1'])
display(df["data1"].groupby(df['key2']).mean())
#非数值字段不参加求均值，如key1字段

**3. 按列表或元组分组**  
分组键还可以是长度和DataFrame行数相同的列表或元组，相当于将列表或元组看做DataFrame的一列，然后将其分组。

【例4-48】按列表或元组分组。

In [ ]:
wlist = ['w','w','y','w','y']
display(df['data1'].groupby(wlist).sum())
df.groupby(wlist).sum()

**4. 按字典分组**  
如果原始的DataFrame中的分组信息很难确定或不存在，可以通过字典结构，定义分组信息。

【例4-49】通过字典作为分组键，分组时各字母不区分大小写。

In [ ]:
df = pd.DataFrame(np.random.normal(size = (6,5)),
    index = ['a','b','c','A','B','c'])
display(df)

In [ ]:
#如果传入的是字典或 Series，则字典或 Series 的值作为分组依据； 
wdict = {'a':'one','A':'one',
    'b':'two','B':'two','c':'three'}
display("分组汇总后的结果为:",
    df.groupby(wdict).sum())

**5. 按函数分组**  
函数作为分组键的原理类似于字典，通过映射关系进行分组，但是函数更加灵活。

【例4-50】通过DataFrame最后一列的数值进行正负分组。

In [ ]:
df = pd.DataFrame(np.random.randn(4,4),
    columns = ("a","b","c","d"))
display(df)

In [ ]:
def judge(x):
    if x>=0:
        return 'a'
    else:
        return 'b'
print(df["d"].groupby(df["d"].map(judge)).sum())

In [ ]:
df = pd.DataFrame(np.random.randn(4,4))
display(df)
# display(df.iloc[0])
# display(df[0:1])

In [ ]:
def judge(x):
    if x>=0:
        return 'a'
    else:
        return 'b'
print(df.groupby(df[3].map(judge)).sum())

### 4.5.2 数据聚合
数据聚合就是对分组后的数据进行计算，产生**标量值**的数据转换过程。

**1. 聚合函数**  
除了之前示例中的mean函数外，常用的聚合运算还有count和sum等。


In [ ]:
%%html
<img src = "tables\table4-4.png", width = 400, height=100>

需要注意的是，在聚合运算中**空值**不参与计算。

**2. 使用agg方法聚合数据**  
agg、aggregate方法都支持对每个分组应用某个函数，包括Python内置函数或自定义函数。  
同时，这两个方法也能够直接对DataFrame进行函数应用操作。  
在正常使用过程中，agg和aggregate函数对DataFrame对象操作的功能基本相同，因此只需掌握一个即可。


1)计算当前数据中的各项统计量  

【例4-51】使用agg求出当前数据对应的统计量。

In [ ]:
data = pd.read_excel('data//testdata.xls')
display(data.head())
# display(data.info())
# display(data.describe())

In [ ]:
#可以用"sum"代替np.sum
display(data[['淋巴细胞计数','白细胞计数']]\
        .agg(["sum",np.mean]))
# data[['淋巴细胞计数','白细胞计数']].sum()

2)计算各字段的不同统计量  

【例4-52】利用agg分别求字段的不同统计量。

In [ ]:
data.agg({'淋巴细胞计数':np.mean,'血小板计数':np.std})

3)计算不同字段的不同数目的统计量  

【例4-53】不同字段统计不同数目的统计量。

In [ ]:
data.agg({'淋巴细胞计数':np.mean,
          '血小板计数':[np.mean,np.std]})

【例4-54】统计不同性别人群的血小板计数均值。

In [ ]:
print(data.groupby('性别')\
      ['血小板计数'].agg(np.mean))

In [ ]:
# 以下语句的效果和上面的一样吗？
# print(data['血小板计数']\
#     .groupby(data['性别']).agg(np.mean))
print(data.groupby('性别')['血小板计数']\
      .agg(np.mean))

In [ ]:
# 效果等价于例4-54吗？
data.groupby('性别')[['血小板计数']].agg(np.mean)
# print(type(data.groupby('性别')[['血小板计数']].agg(np.mean)))

In [ ]:
# 分组后对多个字段进行统计
data.groupby('性别')\
    [['血小板计数','淋巴细胞计数']]\
    .agg([np.mean,np.max])

【例4-55】as_index参数的用法。

In [ ]:
data.groupby(['性别','是否吸烟'])\
    ['血小板计数'].agg(np.mean)

In [ ]:
data.groupby(['性别','是否吸烟'],
    as_index = False)['血小板计数'].agg(np.mean)

### 4.5.3 分组运算
分组运算包含聚合运算，聚合运算是数据转换的特例。

**1. transform方法**  
通过transform方法可以将运算分布到每一行。

【例4-56】transform方法。 

In [ ]:
print(data.groupby(['性别','是否吸烟'])\
    ['血小板计数'].mean())
# print(data.groupby(['性别','是否吸烟'])\
#     ['血小板计数'].agg(np.mean))

In [ ]:
data.groupby(['性别','是否吸烟'])\
    ['血小板计数'].transform('mean')

In [ ]:
data.groupby(['性别','是否吸烟'])\
    ['血小板计数'].transform('mean')

In [ ]:
data['mean']= data.groupby(['性别','是否吸烟'])\
    ['血小板计数'].transform('mean')
display(data)

**2. 使用apply方法聚合数据**  
apply方法类似于agg方法，能够将函数应用于每一列。

【例4-57】数据分组后应用apply统计。

In [ ]:
data.groupby(['性别','是否吸烟'])\
    ['血小板计数'].apply(np.mean)

In [ ]:
data.groupby(['性别','是否吸烟'],group_keys = False)\
    ['血小板计数'].apply(np.mean)

In [ ]:
data.groupby(['性别','是否吸烟'],
    group_keys = False).apply(np.mean)

In [ ]:
data.groupby(['性别','是否吸烟'],
    as_index = False).apply(np.mean)

In [ ]:
mydata = data.groupby(['性别','是否吸烟'],
    group_keys = False).apply(lambda x:x.iloc[[0,1]])
display(mydata)

In [ ]:
df = pd.DataFrame({'key1':list('aaabbbaabb'),
                'key2':[1,2,2,1,2,1,1,2,1,2,],
                'data1':np.random.randn(10),
                'data2':np.random.randn(10)})
display(df)

In [ ]:
a1 = df.groupby(['key1','key2'],
    group_keys=False).apply(lambda x:x.iloc[[0,1]]) 
display(a1)

In [ ]:
a2 = df.groupby(['key1','key2'],
    as_index=False).apply(lambda x:x.iloc[[0,1]])
display(a2)

## 4.6 数据透视表

数据透视表（Pivot Table）是数据分析中常见的工具之一，根据一个或多个键值对数据进行聚合，根据列或行的分组键将数据划分到各个区域。
### 4.6.1 透视表

在Pandas中，除了使用groupby对数据分组聚合实现透视功能外，还可以使用pivot_table函数实现。  

pivot_table函数格式：  
   pivot_table(data, values=None, index=None, columns=None,aggfunc='mean', fill_value=None, margins=False,dropna=True, margins_name='All')


In [ ]:
%%html
<img src = "tables\table4-6.png", width = 500, height=100>

【例4-58】pivot_table默认**计算均值。**

In [ ]:
import pandas as pd
import numpy as np
data = pd.DataFrame({'k1':['a','b','a','a','c','c','b','a','c','a','b','c'],
    'k2':['one','two','three','two','one','one','three','one','two','three','one','two'],
    'w':np.random.rand(12),'y':np.random.randn(12)})
display(data)

In [ ]:
display(data.pivot_table(index = '性别',
        columns = '是否吸烟',aggfunc = np.mean)['血小板计数'])  # 默认求均值

【例4-59】分类汇总并求和。

In [ ]:
data.pivot_table(index = 'k1',
    columns = 'k2',values = 'w',aggfunc = np.sum)

### 4.6.2 交叉表
交叉表是一种特殊的透视表，主要用于计算**分组频率**。

crosstab的格式：  
crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, margins_name='All', dropna=True, normalize=False)

In [ ]:
%%html
<img src = "tables\table4-7.png", width = 500, height=100>

【例4-60】交叉表示例。

In [ ]:
display(data)
pd.crosstab(data.k1,data.k2)

想要在边框处增加汇总项，可以指定margin的值为True。  

【例4-61】带参数margin。

In [ ]:
pd.crosstab(data.k1,data.k2,margins = True)

In [ ]:
display(data)

In [ ]:
pd.crosstab(data.k1,data.k2,data.w,aggfunc=np.average)

## 4.7 Pands 可视化

Pandas中集成了Matplotlib中的基础组件，让绘图更加便捷。

### 4.7.1 线形图  

Pandas库中的Series和DataFrame中都有绘制各类图表的plot方法，默认绘制的都是线形图。  

通过DataFrame对象的plot方法可以为各列绘制一条线，并创建图例。

【例4-62】Series的plot方法绘图。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 当调用matplotlib.pyplot的绘图函数plot()进行绘图的时候，或者生成一个figure画布的时候，
# 可以直接在python console里面生成图像
# %matplotlib inline

s = pd.Series(np.random.normal(size = 10))
plt.ylabel('Value',fontsize=10)
plt.title('this is a test',fontsize=16)
# s.plot()
s.plot(marker='o',linestyle='--',color='r')

In [ ]:
plt.plot(s.index,s.values,
         marker='o',linestyle='--',color='r')

【例4-63】DataFrame的plot方法绘图。

In [ ]:
df = pd.DataFrame({'normal':np.random.normal(size = 50),
    'gamma':np.random.gamma(1,size = 50)})
# display(df)
df["gamma"].plot()

In [ ]:
# df['gamma'].plot()

### 4.7.2 柱状图
柱状图一般用来描述各类别之间的关系。  

在Pandas中绘制柱状图只需在plot函数中加参数kind = ‘bar’，如果类别较多，可以绘制水平柱状图（kind = ‘barh’）。

【例4-64】DataFrame中绘制柱状图。

In [ ]:
stu = {'name':['小明','王芳','赵平','李红','李涵'],
      'sex':['male','female','female','female','male'],
      'year':[1996,1997,1994,1999,1996]}
data = pd.DataFrame(stu)
print(data['sex'].value_counts())
# print(data['sex'].value_counts().\
#       plot(kind = 'bar', rot = 0))
# # print(data['sex'].value_counts().plot())
# # 如果不指定kind='bar'，图形是什么样子
data['year'].plot()

【例4-65】DataFrame数据对象的柱状图。

In [ ]:
df = pd.DataFrame(np.random.randint(1,100,size = (3,3)),
    index = ['one','two','three'],
    columns = ['I1','I2','I3'])
display(df)
df.plot(kind = 'bar')
df.plot(kind = 'barh')

### 4.7.3 直方图和密度图  
直方图用于频率分布，y轴为数值或比率。绘制直方图，可以观察数据值的大致分布规律。pandas中的直方图可以通过hist方法绘制。  

核密度估计是对真实密度的估计，其过程是将数据的分布近似为一组核（如正态分布）。通过plot函数的**kind = ‘kde’**可以进行绘制。


【例4-66】pandas中直方图绘制。

In [ ]:
wy = pd.Series(np.random.normal(size = 80000))
# display(wy)
wy.hist(bins = 100,grid = True)

【例4-67】panas中密度图的绘制。

In [ ]:
wy = pd.Series(np.random.normal(size = 8000))
wy.plot(kind = 'kde')

### 4.7.4 散点图  

散点图主要用来表现数据之间的规律

通过plot函数的kind = 'scatter'可以进行绘制。


【例4-68】pandas绘制散点图。

In [ ]:
wd = pd.DataFrame(np.arange(10),columns = ['A'])
wd['B'] = 2*wd['A']+4
wd.plot(kind = 'scatter',x = 'A',y = 'B')
plt.savefig("./test.png")

In [ ]:
%%html
<img src = "tables\usage_describe.png", width=300, height=100>